## Delivery Duration Prediction

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

## Step 1: Gathering the data

In [ ]:
import pandas as pd 
path = r"G:\Machine Learning\Projects\Dataset for project.csv"
df = pd.read_csv(path)
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


## Step 2: Data Understanding and basic data quality checks

In [3]:
df.shape

(197428, 16)

In [4]:
df.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [6]:
## Missing values
missing = df.isna().sum()
missing[missing>0]

market_id                                         987
actual_delivery_time                                7
store_primary_category                           4760
order_protocol                                    995
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_store_to_consumer_driving_duration      526
dtype: int64

In [82]:
df.nunique()

market_id                                           6
created_at                                      46077
actual_delivery_time                            46088
store_id                                         6743
store_primary_category                             74
order_protocol                                      7
total_items                                        57
subtotal                                         8368
num_distinct_items                                 20
min_item_price                                   2312
max_item_price                                   2652
total_onshift_dashers                             172
total_busy_dashers                                159
total_outstanding_orders                          281
estimated_order_place_duration                     98
estimated_store_to_consumer_driving_duration     1336
total_delivery_time                             10060
delivery_duration_sec                           10060
dtype: int64

In [7]:
## Check the duplicates
df.duplicated().sum()

np.int64(0)

In [8]:
## Drop the dupolicates
df = df.drop_duplicates()

#### There are no duplicates rows now, but in the future, duplicate rows might appear, so we will need to drop them.

## Datatype Conversion 

In [93]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])
df['delivery_duration_sec'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()

In [94]:
print(df['delivery_duration_sec'].dtype)

float64


#### Separate X and Y features
    Y: total_delivery_time
    X: all remaining features 

In [59]:
X = df.drop(columns=['delivery_duration_sec','created_at','actual_delivery_time'])
Y = df[['delivery_duration_sec']]

In [60]:
X.head()

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,total_delivery_time
0,1.0,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0
1,2.0,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0
2,3.0,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0
3,3.0,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0
4,3.0,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0


In [61]:
X.shape

(197428, 15)

In [62]:
Y.head()

,delivery_duration_sec
0,3779.0
1,4024.0
2,1781.0
3,3075.0
4,2390.0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 17 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   market_id                                     196441 non-null  float64       
 1   created_at                                    197428 non-null  datetime64[ns]
 2   actual_delivery_time                          197421 non-null  datetime64[ns]
 3   store_id                                      197428 non-null  int64         
 4   store_primary_category                        192668 non-null  object        
 5   order_protocol                                196433 non-null  float64       
 6   total_items                                   197428 non-null  int64         
 7   subtotal                                      197428 non-null  int64         
 8   num_distinct_items                            197428 n

## Step 3: Data Preprocessing and Data Cleaning

In [114]:
cat = list(X.columns[X.dtypes == 'object'])
con = list(X.columns[X.dtypes == 'int64']) or (X.columns[X.dtypes=='float64'])
time = list(df.columns[df.dtypes == 'datetime64[ns]'])

In [115]:
print(time)

['created_at', 'actual_delivery_time']


In [99]:
print(con)

['store_id', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'estimated_order_place_duration']


In [129]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder

In [131]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),  # data cleaning
    OrdinalEncoder()
    # OneHotEncoder(handle_unknown='ignore',sparse_output=False)  # feature scaling 
)

In [132]:
con_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),  # data cleaning
    StandardScaler()
)

In [133]:
time_pipe = 'passthrough'

In [ ]:
# Handle datetime column using Forward fill
df[['created_at','actual_delivery_time']] = df[['created_at','actual_delivery_time']].fillna(method='ffill')

In [134]:
pre = ColumnTransformer([
    ('cat',cat_pipe,cat),
    ('con',con_pipe,con),
    ('time',time_pipe,time)
    ]).set_output(transform='pandas')

In [136]:
pre

,transformers,"[('cat', ...), ('con', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None


In [137]:
X_pre = pre.fit_transform(df)
X_pre.head()

,cat__store_primary_category,con__store_id,con__total_items,con__subtotal,con__num_distinct_items,con__min_item_price,con__max_item_price,con__estimated_order_place_duration,time__created_at,time__actual_delivery_time
0,4.0,-0.820802,0.301368,0.416145,0.815340,-0.247527,0.142210,1.524747,2025-10-27 22:24:17,2025-10-27 23:27:16
1,47.0,0.947893,-0.823686,-0.429124,-1.024867,1.367300,0.430528,1.524747,2025-10-27 21:49:25,2025-10-27 22:56:29
2,4.0,0.947893,-0.823686,-0.429124,-1.024867,2.325086,1.325928,1.524747,2025-10-27 20:39:28,2025-10-27 21:09:09
3,4.0,0.947893,1.051404,2.313474,1.428742,-0.165158,1.146848,1.524747,2025-10-27 21:21:45,2025-10-27 22:13:00
4,4.0,0.947893,-0.073650,0.667915,0.201937,0.792628,0.788688,1.524747,2025-10-27 02:40:36,2025-10-27 03:20:26


In [120]:
X_pre.tail()

,cat__store_primary_category_afghan,cat__store_primary_category_african,cat__store_primary_category_alcohol,cat__store_primary_category_alcohol-plus-food,cat__store_primary_category_american,cat__store_primary_category_argentine,cat__store_primary_category_asian,cat__store_primary_category_barbecue,cat__store_primary_category_belgian,cat__store_primary_category_brazilian,...,cat__store_primary_category_vietnamese,con__store_id,con__total_items,con__subtotal,con__num_distinct_items,con__min_item_price,con__max_item_price,con__estimated_order_place_duration,time__created_at,time__actual_delivery_time
197423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.279772,-0.073650,-0.709418,0.201937,-0.653628,-0.914362,-0.638568,2025-10-27 00:19:41,2025-10-27 01:24:48
197424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.279772,1.051404,0.179733,0.815340,-0.538694,-0.599181,-0.638568,2025-10-27 00:01:59,2025-10-27 00:58:22
197425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.279772,0.676386,-0.464229,0.201937,-0.739829,-1.362061,-0.638568,2025-10-27 04:46:08,2025-10-27 05:36:16
197426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.048449,-0.823686,-0.826801,-1.024867,-0.289670,-1.118513,1.524747,2025-10-27 18:18:15,2025-10-27 19:23:22
197427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.048449,0.301368,-0.042418,0.815340,-0.500383,-0.733491,1.524747,2025-10-27 19:24:33,2025-10-27 20:01:41


In [138]:
sel_col = []
for i in X_pre:
    cols = i.split('__')[-1]
    sel_col.append(cols)
sel_col

['store_primary_category',
 'store_id',
 'total_items',
 'subtotal',
 'num_distinct_items',
 'min_item_price',
 'max_item_price',
 'estimated_order_place_duration',
 'created_at',
 'actual_delivery_time']

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit (X_pre,Y)

ValueError: Expected 2D array, got 1D array instead:
array=['store_primary_category' 'store_id' 'total_items' 'subtotal'
 'num_distinct_items' 'min_item_price' 'max_item_price'
 'estimated_order_place_duration' 'created_at' 'actual_delivery_time'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.